In [3]:
# PyTorch 및 필요한 라이브러리 설치
!pip install torch torchvision torchaudio
!pip install scikit-learn matplotlib seaborn pandas numpy tqdm

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import pickle
import os
import requests
from io import StringIO
import json
from collections import defaultdict
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# CUDA 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" 사용 디바이스: {device}")

 사용 디바이스: cpu


In [23]:
# Spotify 데이터셋 다운로드
# ========================================

def download_spotify_dataset():
    """Spotify 데이터셋 다운로드"""
    url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv"
    filename = "spotify_songs.csv"
    
    if not os.path.exists(filename):
        print(" Spotify 데이터셋을 다운로드 중...")
        try:
            response = requests.get(url)
            response.raise_for_status()
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(response.text)
            print(" 다운로드 완료!")
        except Exception as e:
            print(f" 다운로드 실패: {e}")
            return None
    else:
        print(" 데이터셋이 이미 존재합니다.")
    
    return filename

# 데이터셋 다운로드 실행
dataset_file = download_spotify_dataset()
print(f"데이터셋 파일: {dataset_file}")

 데이터셋이 이미 존재합니다.
데이터셋 파일: spotify_songs.csv


In [24]:
#  PyTorch 신경망 모델 정의
# ========================================

class MusicEmotionClassifier(nn.Module):
    """음악 감정 분류를 위한 신경망 모델"""
    
    def __init__(self, input_size, hidden_sizes=[256, 128, 64], num_classes=5, dropout_rate=0.3):
        super(MusicEmotionClassifier, self).__init__()
        
        layers = []
        prev_size = input_size
        
        for i, hidden_size in enumerate(hidden_sizes):
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate if i < len(hidden_sizes)-1 else dropout_rate/2)
            ])
            prev_size = hidden_size
        
        # 출력층
        layers.extend([
            nn.Linear(prev_size, num_classes),
            nn.Softmax(dim=1)
        ])
        
        self.network = nn.Sequential(*layers)
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            nn.init.zeros_(module.bias)
    
    def forward(self, x):
        return self.network(x)

class ValencePredictor(nn.Module):
    """Valence 값 예측을 위한 회귀 모델"""
    
    def __init__(self, input_size, hidden_sizes=[128, 64, 32], dropout_rate=0.2):
        super(ValencePredictor, self).__init__()
        
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        
        # 출력층 (0-1 범위)
        layers.extend([
            nn.Linear(prev_size, 1),
            nn.Sigmoid()
        ])
        
        self.network = nn.Sequential(*layers)
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            nn.init.zeros_(module.bias)
    
    def forward(self, x):
        return self.network(x)

print(" 신경망 모델 정의 완료!")


 신경망 모델 정의 완료!


In [25]:
# ========================================
#  음악 감정 분석 및 추천 시스템 (1부)
# ========================================

class SpotifyEmotionRecommendationSystem:
    def __init__(self, random_seed=42):
        """시스템 초기화"""
        
        # 시드 설정
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)
        
        self.device = device
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.df = None
        self.emotion_classifier = None
        self.valence_predictor = None
        
        # 감정 카테고리 정의 (valence 기반)
        self.emotion_categories = {
            'very_sad': {'valence_range': (0.0, 0.2), 'energy_weight': 0.3, 'label': '매우 슬픔'},
            'sad': {'valence_range': (0.2, 0.4), 'energy_weight': 0.4, 'label': '슬픔'},
            'calm': {'valence_range': (0.4, 0.6), 'energy_weight': 0.5, 'label': '평온'},
            'happy': {'valence_range': (0.6, 0.8), 'energy_weight': 0.6, 'label': '행복'},
            'very_happy': {'valence_range': (0.8, 1.0), 'energy_weight': 0.7, 'label': '매우 행복'}
        }
        
        # 세분화된 감정 매핑 (일기 감정라벨과 연동 준비)
        self.emotion_mapping = {
            # 매우 부정적 감정
            '절망': 'very_sad', '우울': 'very_sad', '좌절': 'very_sad', 
            '비참': 'very_sad', '암울': 'very_sad',
            
            # 부정적 감정
            '슬픔': 'sad', '외로움': 'sad', '그리움': 'sad', 
            '아쉬움': 'sad', '후회': 'sad', '실망': 'sad',
            
            # 중립적 감정
            '평온': 'calm', '차분': 'calm', '안정': 'calm', '보통': 'calm',
            '무난': 'calm', '여유': 'calm', '편안': 'calm',
            
            # 긍정적 감정
            '기쁨': 'happy', '행복': 'happy', '즐거움': 'happy', 
            '만족': 'happy', '고마움': 'happy', '사랑': 'happy',
            
            # 매우 긍정적 감정
            '환희': 'very_happy', '황홀': 'very_happy', '열광': 'very_happy',
            '신남': 'very_happy', '흥분': 'very_happy', '설렘': 'very_happy'
        }
        
        print(" Spotify 음악 감정 추천 시스템 초기화 완료!")
        print(f" 감정 카테고리: {len(self.emotion_categories)}개")
        print(f" 감정 매핑: {len(self.emotion_mapping)}개")

# 시스템 초기화 테스트
system = SpotifyEmotionRecommendationSystem()
print("\n 시스템 객체 생성 완료!")

 Spotify 음악 감정 추천 시스템 초기화 완료!
 감정 카테고리: 5개
 감정 매핑: 30개

 시스템 객체 생성 완료!


In [26]:
 ========================================
#  데이터 로드 및 전처리 메서드 추가
# ========================================

def load_and_preprocess_data(self, file_path=None):
    """데이터 로드 및 전처리"""
    try:
        # 파일 경로가 없으면 다운로드
        if file_path is None:
            file_path = download_spotify_dataset()
            if file_path is None:
                return None
        
        print(" 데이터 로드 중...")
        self.df = pd.read_csv(file_path)
        print(f" 원본 데이터: {len(self.df):,} 곡")
        
        # 필요한 컬럼 확인
        required_cols = ['valence', 'energy', 'danceability', 'acousticness']
        missing_cols = [col for col in required_cols if col not in self.df.columns]
        
        if missing_cols:
            print(f" 필수 컬럼 누락: {missing_cols}")
            return None
        
        # 데이터 정리
        print(" 데이터 전처리 중...")
        
        # 결측치 제거
        original_len = len(self.df)
        self.df = self.df.dropna(subset=required_cols)
        print(f" 결측치 제거: {original_len - len(self.df):,} 곡 제거")
        
        # 이상치 제거 (0-1 범위 벗어난 값)
        for col in ['valence', 'energy', 'danceability', 'acousticness']:
            if col in self.df.columns:
                before_len = len(self.df)
                self.df = self.df[(self.df[col] >= 0) & (self.df[col] <= 1)]
                if len(self.df) < before_len:
                    print(f" {col} 이상치 제거: {before_len - len(self.df)} 곡")
        
        # 감정 라벨 생성
        print(" 감정 라벨 생성 중...")
        self.df['emotion_category'] = self.df.apply(self._classify_emotion, axis=1)
        
        # 감정별 분포 출력
        print("\n 감정별 분포:")
        emotion_counts = self.df['emotion_category'].value_counts()
        for emotion, count in emotion_counts.items():
            label = self.emotion_categories[emotion]['label']
            print(f"  {label} ({emotion}): {count:,}곡 ({count/len(self.df)*100:.1f}%)")
        
        # 데이터 시각화
        self._visualize_data_distribution()
        
        print(f"\n 전처리 완료: {len(self.df):,} 곡")
        return self.df
        
    except Exception as e:
        print(f" 데이터 로드 실패: {e}")
        return None

def _classify_emotion(self, row):
    """valence와 energy 기반 감정 분류"""
    valence = row['valence']
    energy = row.get('energy', 0.5)  # 기본값
    
    # valence 기반 1차 분류
    for emotion, config in self.emotion_categories.items():
        v_min, v_max = config['valence_range']
        if v_min <= valence < v_max:
            # energy 가중치 적용한 최종 판단
            energy_threshold = config['energy_weight']
            if energy >= energy_threshold or emotion in ['very_sad', 'sad']:
                return emotion
    
    # 기본값
    return 'calm'

# 클래스에 메서드 추가
SpotifyEmotionRecommendationSystem.load_and_preprocess_data = load_and_preprocess_data
SpotifyEmotionRecommendationSystem._classify_emotion = _classify_emotion

print(" 데이터 전처리 메서드 추가 완료!")


SyntaxError: invalid syntax (<ipython-input-26-12b4ab87e578>, line 1)

In [27]:
# 🎵 통합 음악 감정 분류 및 추천 시스템
# ========================================

class MusicEmotionRecommendationSystem:
    def __init__(self, random_seed=42):
        """음악 감정 분류 및 추천 시스템 초기화"""
        
        # 시드 설정
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)
        
        self.device = device
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.df = None
        self.emotion_classifier = None
        self.valence_regressor = None
        
        # 감정 카테고리 정의 (valence와 energy 기반)
        self.emotion_categories = {
            'sad': {'valence': (0.0, 0.3), 'energy': (0.0, 0.5)},
            'calm': {'valence': (0.3, 0.7), 'energy': (0.0, 0.6)},
            'happy': {'valence': (0.7, 1.0), 'energy': (0.5, 1.0)},
            'angry': {'valence': (0.0, 0.4), 'energy': (0.7, 1.0)},
            'energetic': {'valence': (0.6, 1.0), 'energy': (0.8, 1.0)}
        }
        
        # 일기 감정 → 음악 감정 매핑
        self.diary_to_music_emotion = {
            # 긍정적 감정
            '기쁨': 'happy', '행복': 'happy', '즐거움': 'happy', '만족': 'happy',
            '사랑': 'happy', '고마움': 'happy', '희망': 'happy', '설렘': 'happy',
            
            # 활동적 감정
            '신남': 'energetic', '활기': 'energetic', '흥분': 'energetic', 
            '에너지': 'energetic', '자신감': 'energetic', '의욕': 'energetic',
            
            # 부정적 감정
            '슬픔': 'sad', '우울': 'sad', '눈물': 'sad', '외로움': 'sad',
            '그리움': 'sad', '아쉬움': 'sad', '후회': 'sad', '실망': 'sad',
            
            # 분노 관련 감정
            '화남': 'angry', '분노': 'angry', '짜증': 'angry', '화': 'angry',
            '불만': 'angry', '스트레스': 'angry', '답답함': 'angry',
            
            # 평온한 감정
            '평온': 'calm', '차분': 'calm', '안정': 'calm', '편안': 'calm',
            '보통': 'calm', '그냥': 'calm', '무난': 'calm', '여유': 'calm',
            '피곤': 'calm', '지침': 'calm'
        }
        
        print("음악 감정 분류 및 추천 시스템 초기화 완료!")
    
    def load_spotify_data(self, file_path):
        """Spotify 데이터셋 로드 및 전처리"""
        try:
            self.df = pd.read_csv(file_path)
            print(f" 데이터 로드 완료: {len(self.df):,} 곡")
            
            # 데이터 정보 출력
            print(f" 데이터셋 컬럼: {list(self.df.columns)}")
            
            # 필요한 컬럼 확인 및 매핑
            column_mapping = {
                'track_name': 'name',
                'track_artist': 'artists',
                'track_album_name': 'album',
                'track_popularity': 'popularity'
            }
            
            for old_col, new_col in column_mapping.items():
                if old_col in self.df.columns and new_col not in self.df.columns:
                    self.df[new_col] = self.df[old_col]
            
            # 필요한 오디오 특성 확인
            audio_features = ['valence', 'energy', 'danceability', 'acousticness', 
                             'instrumentalness', 'liveness', 'speechiness']
            missing_features = [f for f in audio_features if f not in self.df.columns]
            
            if missing_features:
                print(f" 누락된 오디오 특성: {missing_features}")
                return None
            
            # 결측치 처리
            print(f" 결측치 제거 전: {len(self.df):,} 곡")
            self.df = self.df.dropna(subset=audio_features)
            print(f" 결측치 제거 후: {len(self.df):,} 곡")
            
            # valence와 energy 값 범위 확인 및 정규화
            for feature in ['valence', 'energy']:
                if self.df[feature].max() > 1:
                    print(f"⚠️ {feature} 값을 0-1 범위로 정규화합니다.")
                    self.df[feature] = self.df[feature] / self.df[feature].max()
            
            # 감정 라벨 생성
            print(" 감정 라벨 생성 중...")
            self.df['emotion_label'] = self.df.apply(
                lambda row: self._classify_emotion_by_valence_energy(
                    row['valence'], row['energy']
                ), axis=1
            )
            
            # 감정별 분포 출력
            print(f"\n 감정별 분포:")
            emotion_counts = self.df['emotion_label'].value_counts()
            for emotion, count in emotion_counts.items():
                print(f"  {emotion}: {count:,}곡 ({count/len(self.df)*100:.1f}%)")
            
            # 데이터 시각화
            self._visualize_emotion_distribution()
            
            return self.df
            
        except Exception as e:
            print(f" 데이터 로드 실패: {e}")
            return None
    
    def _classify_emotion_by_valence_energy(self, valence, energy):
        """valence와 energy 값으로 감정 분류"""
        for emotion, ranges in self.emotion_categories.items():
            if (ranges['valence'][0] <= valence <= ranges['valence'][1] and 
                ranges['energy'][0] <= energy <= ranges['energy'][1]):
                return emotion
        return 'calm'  # 기본값
    
    def _visualize_emotion_distribution(self):
        """감정 분포 시각화"""
        plt.figure(figsize=(15, 5))
        
        # 1. 감정별 분포 파이차트
        plt.subplot(1, 3, 1)
        emotion_counts = self.df['emotion_label'].value_counts()
        plt.pie(emotion_counts.values, labels=emotion_counts.index, autopct='%1.1f%%')
        plt.title('감정별 곡 분포')
        
        # 2. Valence vs Energy 산점도
        plt.subplot(1, 3, 2)
        colors = {'sad': 'blue', 'calm': 'green', 'happy': 'yellow', 
                 'angry': 'red', 'energetic': 'orange'}
        
        for emotion in self.emotion_categories.keys():
            emotion_data = self.df[self.df['emotion_label'] == emotion]
            if len(emotion_data) > 0:
                plt.scatter(emotion_data['valence'], emotion_data['energy'], 
                          c=colors.get(emotion, 'gray'), label=emotion, alpha=0.6, s=1)
        
        plt.xlabel('Valence')
        plt.ylabel('Energy')
        plt.title('Valence vs Energy by Emotion')
        plt.legend()
        
        # 3. Valence 분포 히스토그램
        plt.subplot(1, 3, 3)
        plt.hist(self.df['valence'], bins=30, alpha=0.7, edgecolor='black')
        plt.xlabel('Valence')
        plt.ylabel('Frequency')
        plt.title('Valence 분포')
        
        plt.tight_layout()
        plt.show()
    
    def prepare_features_and_labels(self, for_regression=False):
        """모델 훈련을 위한 특성과 라벨 준비"""
        if self.df is None:
            print(" 데이터를 먼저 로드해주세요.")
            return None, None
        
        # 오디오 특성 선택
        if for_regression:
            # 회귀용: valence 제외
            audio_features = ['energy', 'danceability', 'acousticness', 
                             'instrumentalness', 'liveness', 'speechiness']
            target = 'valence'
        else:
            # 분류용: 모든 특성 포함
            audio_features = ['valence', 'energy', 'danceability', 'acousticness', 
                             'instrumentalness', 'liveness', 'speechiness']
            target = 'emotion_label'
        
        # 실제 존재하는 특성만 선택
        available_features = [f for f in audio_features if f in self.df.columns]
        
        if len(available_features) < 2:
            print(f" 충분한 오디오 특성이 없습니다. 사용 가능: {available_features}")
            return None, None
        
        print(f" 사용할 특성: {available_features}")
        
        # 특성 및 타겟 추출
        X = self.df[available_features].values
        
        if for_regression:
            y = self.df[target].values
        else:
            y = self.df[target].values
            y = self.label_encoder.fit_transform(y)
        
        # 특성 정규화
        X_scaled = self.scaler.fit_transform(X)
        
        return X_scaled, y
    
    def train_emotion_classifier(self, test_size=0.2, batch_size=256, epochs=50, lr=0.001):
        """감정 분류 모델 훈련"""
        print(" 감정 분류 모델 훈련 시작...")
        
        X, y = self.prepare_features_and_labels(for_regression=False)
        if X is None or y is None:
            return False
        
        # 훈련/테스트 분할
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42, stratify=y
        )
        
        # PyTorch 텐서 변환
        X_train_tensor = torch.FloatTensor(X_train).to(self.device)
        y_train_tensor = torch.LongTensor(y_train).to(self.device)
        X_test_tensor = torch.FloatTensor(X_test).to(self.device)
        y_test_tensor = torch.LongTensor(y_test).to(self.device)
        
        # 데이터 로더
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        # 모델 초기화
        input_size = X.shape[1]
        num_classes = len(self.label_encoder.classes_)
        self.emotion_classifier = MusicEmotionNet(input_size, num_classes=num_classes).to(self.device)
        
        # 손실 함수 및 최적화기
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.emotion_classifier.parameters(), lr=lr, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
        
        # 훈련
        train_losses, train_accuracies = [], []
        
        self.emotion_classifier.train()
        for epoch in tqdm(range(epochs), desc="분류 모델 훈련"):
            epoch_loss, correct, total = 0, 0, 0
            
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = self.emotion_classifier(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                
                epoch_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += batch_y.size(0)
                correct += (predicted == batch_y).sum().item()
            
            avg_loss = epoch_loss / len(train_loader)
            accuracy = 100 * correct / total
            train_losses.append(avg_loss)
            train_accuracies.append(accuracy)
            scheduler.step(avg_loss)
            
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{epochs}] Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
        

In [28]:
# ========================================
#  데이터 시각화 메서드
# ========================================

def _visualize_data_distribution(self):
    """데이터 분포 시각화"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 감정별 분포
    emotion_counts = self.df['emotion_category'].value_counts()
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']
    
    axes[0,0].pie(emotion_counts.values, labels=[self.emotion_categories[e]['label'] for e in emotion_counts.index], 
                 autopct='%1.1f%%', colors=colors)
    axes[0,0].set_title('감정별 곡 분포', fontsize=14, fontweight='bold')
    
    # 2. Valence vs Energy 산점도
    emotion_colors = {
        'very_sad': '#FF6B6B', 'sad': '#FF8E8E', 'calm': '#4ECDC4',
        'happy': '#96CEB4', 'very_happy': '#FFEAA7'
    }
    
    for emotion in self.emotion_categories.keys():
        emotion_data = self.df[self.df['emotion_category'] == emotion]
        if len(emotion_data) > 0:
            axes[0,1].scatter(emotion_data['valence'], emotion_data['energy'], 
                            c=emotion_colors.get(emotion, 'gray'), 
                            label=self.emotion_categories[emotion]['label'],
                            alpha=0.6, s=10)
    
    axes[0,1].set_xlabel('Valence (감정가)')
    axes[0,1].set_ylabel('Energy (에너지)')
    axes[0,1].set_title('Valence vs Energy 분포')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Valence 히스토그램
    axes[1,0].hist(self.df['valence'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[1,0].axvline(self.df['valence'].mean(), color='red', linestyle='--', 
                     label=f'평균: {self.df["valence"].mean():.3f}')
    axes[1,0].set_xlabel('Valence')
    axes[1,0].set_ylabel('빈도')
    axes[1,0].set_title('Valence 분포')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. 주요 오디오 특성 박스플롯
    audio_features = ['valence', 'energy', 'danceability', 'acousticness']
    available_features = [f for f in audio_features if f in self.df.columns]
    
    box_data = [self.df[feature].values for feature in available_features]
    bp = axes[1,1].boxplot(box_data, labels=available_features, patch_artist=True)
    
    # 박스 색상 설정
    box_colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
    for patch, color in zip(bp['boxes'], box_colors[:len(available_features)]):
        patch.set_facecolor(color)
    
    axes[1,1].set_title('오디오 특성 분포')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 클래스에 메서드 추가
SpotifyEmotionRecommendationSystem._visualize_data_distribution = _visualize_data_distribution

print(" 시각화 메서드 추가 완료!")

 시각화 메서드 추가 완료!


In [31]:
# ========================================
#  모델 훈련 관련 메서드
# ========================================

def prepare_training_data(self, test_size=0.2):
    """모델 훈련용 데이터 준비"""
    if self.df is None:
        print(" 데이터를 먼저 로드해주세요.")
        return None
    
    # 특성 선택
    feature_cols = ['valence', 'energy', 'danceability', 'acousticness', 
                   'instrumentalness', 'liveness', 'speechiness', 'loudness', 'tempo']
    available_features = [col for col in feature_cols if col in self.df.columns]
    
    print(f" 사용 특성: {available_features}")
    
    # 특성 행렬
    X = self.df[available_features].values
    
    # 타겟 라벨
    y = self.df['emotion_category'].values
    y_encoded = self.label_encoder.fit_transform(y)
    
    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=test_size, random_state=42, stratify=y_encoded
    )
    
    # 특성 정규화
    X_train_scaled = self.scaler.fit_transform(X_train)
    X_test_scaled = self.scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

def _evaluate_model(self, X_test, y_test):
    """모델 평가"""
    self.emotion_classifier.eval()
    with torch.no_grad():
        outputs = self.emotion_classifier(X_test)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test.cpu(), predicted.cpu())
        
        print(f" 테스트 정확도: {accuracy:.4f}")
        print("\n 상세 분류 리포트:")
        print(classification_report(
            y_test.cpu(), 
            predicted.cpu(),
            target_names=[self.emotion_categories[cat]['label'] 
                         for cat in self.label_encoder.classes_]
        ))
        
        # Confusion Matrix 시각화
        cm = confusion_matrix(y_test.cpu(), predicted.cpu())
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=[self.emotion_categories[cat]['label'] for cat in self.label_encoder.classes_],
                   yticklabels=[self.emotion_categories[cat]['label'] for cat in self.label_encoder.classes_])
        plt.title('감정 분류 Confusion Matrix')
        plt.xlabel('예측')
        plt.ylabel('실제')
        plt.show()
        
        return accuracy

def _plot_training_history(self, losses, accuracies):
    """훈련 과정 시각화"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss
    ax1.plot(losses, color='red', linewidth=2)
    ax1.set_title('Training Loss', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.grid(True, alpha=0.3)
    
    # Accuracy
    ax2.plot(accuracies, color='blue', linewidth=2)
    ax2.set_title('Training Accuracy', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 클래스에 메서드 추가
SpotifyEmotionRecommendationSystem.prepare_training_data = prepare_training_data
SpotifyEmotionRecommendationSystem._evaluate_model = _evaluate_model
SpotifyEmotionRecommendationSystem._plot_training_history = _plot_training_history

print(" 모델 훈련 관련 메서드 추가 완료!")

 모델 훈련 관련 메서드 추가 완료!


In [30]:
# ========================================
# 메인 훈련 메서드
# ========================================

def train_emotion_classifier(self, epochs=100, batch_size=256, lr=0.001):
    """감정 분류 모델 훈련"""
    print("감정 분류 모델 훈련 시작...")
    
    # 데이터 준비
    data = self.prepare_training_data()
    if data is None:
        return False
    
    X_train, X_test, y_train, y_test = data
    
    # PyTorch 텐서 변환
    X_train_tensor = torch.FloatTensor(X_train).to(self.device)
    y_train_tensor = torch.LongTensor(y_train).to(self.device)
    X_test_tensor = torch.FloatTensor(X_test).to(self.device)
    y_test_tensor = torch.LongTensor(y_test).to(self.device)
    
    # 데이터 로더
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # 모델 초기화
    input_size = X_train.shape[1]
    num_classes = len(self.label_encoder.classes_)
    self.emotion_classifier = MusicEmotionClassifier(
        input_size=input_size, 
        num_classes=num_classes
    ).to(self.device)
    
    # 손실 함수와 최적화기
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(self.emotion_classifier.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=15, factor=0.5)
    
    # 훈련
    train_losses, train_accuracies = [], []
    
    for epoch in tqdm(range(epochs), desc="모델 훈련"):
        self.emotion_classifier.train()
        epoch_loss, correct, total = 0, 0, 0
        
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = self.emotion_classifier(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            
            # Gradient Clipping
            torch.nn.utils.clip_grad_norm_(self.emotion_classifier.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            epoch_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
        
        avg_loss = epoch_loss / len(train_loader)
        accuracy = 100 * correct / total
        train_losses.append(avg_loss)
        train_accuracies.append(accuracy)
        scheduler.step(avg_loss)
        
        # 주기적 출력
        if (epoch + 1) % 20 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    # 테스트 평가
    print("\n테스트 데이터 평가 중...")
    test_accuracy = self._evaluate_model(X_test_tensor, y_test_tensor)
    
    # 훈련 과정 시각화
    self._plot_training_history(train_losses, train_accuracies)
    
    # 모델 저장
    self.save_models()
    
    return True

# 클래스에 메서드 추가
